# PreProcessing Images to generate a Dataset

In [15]:
!pip install google-cloud-storage tensorflow --quiet

  Could not find a version that satisfies the requirement cv2 (from versions: )
No matching distribution found for cv2


In [1]:
import os

from google.cloud import storage
import tensorflow as tf
from sklearn.preprocessing import LabelBinarizer
from datetime import datetime as dt
from multiprocessing import Pool
from skimage.io import imread
from tensorflow.keras.applications import vgg16
import numpy as np

from tensorflow.python.lib.io import file_io
import _pickle as pickle

In [2]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/home/jovyan/work/image-classifier/bigdata-217213-55b1dfc31b66.json'

In [3]:
basepath = 'gs://bigdata-allanbatista-com-br/trainer/{}/'.format(dt.now().strftime('%Y%m%d_%H%M%S'))
basepath

'gs://bigdata-allanbatista-com-br/trainer/20181110_183306/'

In [4]:
gs = storage.Client()
bucket = gs.bucket('bigdata-allanbatista-com-br')

def list_blobs(pattern):
    return [blob.name for blob in bucket.list_blobs(prefix=pattern)]

In [5]:
def list_images_with_labels(pattern):
    x = []
    y = []
    for path in list_blobs(pattern):
        x.append(path)
        y.append(path.split("/")[2])
    
    return x, y

In [6]:
paths, labels = list_images_with_labels("dataset/train")

In [7]:
binarizer = LabelBinarizer()
y_train = binarizer.fit_transform(labels)

In [8]:
with file_io.FileIO('{}binarizer.pickle'.format(basepath), 'wb+') as f:
    f.write(pickle.dumps(binarizer))

In [22]:
# filename = 'gs://bigdata-allanbatista-com-br/{}'.format(paths[0])
# with file_io.FileIO(filename, 'rb') as file:
#     image = imread(file)

# np.reshape(np.array(image), (len))

array([ 56,  64,  67, ..., 120, 129, 112], dtype=uint8)

In [51]:
chunck_size = 10
fff = None

def create_record(features, label):
    features = tf.train.Features(feature={
        'label': tf.train.Feature(int64_list=tf.train.Int64List(value=label)),
        'features': tf.train.Feature(bytes_list=tf.train.BytesList(value=[features.tostring()]))
    })
    
    return tf.train.Example(features=features)

def read_image(path):
    filename = 'gs://bigdata-allanbatista-com-br/{}'.format(path)

    with file_io.FileIO(filename, 'rb') as file:
        image = imread(file)
    
    return image
    
def create_and_write_record(data, x, y, dest_path):
    i = data[0]
    index = data[1]
    start = dt.now()
    filename = "{}{}.tfrecord".format(dest_path, str(index).zfill(5))
    
    with Pool(10) as p:
        images = np.array(p.map(read_image, x[i:i+chunck_size]))
    
    features = np.reshape(images, (len(images), 256 * 256 * 3))

    print(features.shape)
    
    print("writing in doc")
    with tf.python_io.TFRecordWriter(filename) as writer:
        for feature, label in zip(images, y[i:i+chunck_size]):
            record = create_record(feature, label)
            writer.write(record.SerializeToString())            

    print("diff %ds: %s" % ((dt.now() - start).total_seconds(), filename))

    
def create_dataset_to_gs(x, y, dest_path):
    chunck_i = list(range(0, len(y), chunck_size))
    chunck_index = list(range(len(chunck_i)))

    for data in list(zip(chunck_i, chunck_index)):
        print("create first chunck")
        create_and_write_record(data, x, y, dest_path)
        break

In [53]:
create_dataset_to_gs(paths, y_train, "{}trainset_simple/".format(basepath))

create first chunck
(10, 196608)
writing in doc
diff 2s: gs://bigdata-allanbatista-com-br/trainer/20181110_183306/trainset_simple/00000.tfrecord


In [44]:
reshapeds

(196608,)